# **Chapter 25: Appium**

---

## **25.1 Introduction to Appium**

### **What is Appium?**

Appium is an open-source, cross-platform test automation framework for mobile applications. It drives native, hybrid, and mobile web apps using the WebDriver protocol, the same standard used for web browser automation (Selenium).

**Official Definition:**
> "Appium is an open-source automation framework for testing native, hybrid, and mobile web applications. It drives iOS, Android, and Windows apps using the WebDriver protocol."

**Why Appium is the Industry Standard:**

1. **Cross-Platform:** Single API for iOS and Android (write once, run on both)
2. **Language Agnostic:** Supports Python, Java, JavaScript, C#, Ruby
3. **No App Modification:** Tests run against compiled apps (APK/IPA), no source code changes or recompilation needed
4. **Standard Protocol:** Uses W3C WebDriver standard, ensuring compatibility and future-proofing
5. **Open Source:** Free, with strong community backing (hosted by the OpenJS Foundation)
6. **Flexibility:** Works with real devices, emulators, simulators, and cloud device farms

**Appium vs. Native Frameworks:**

| Feature | Appium | Espresso (Android) | XCUITest (iOS) |
|---------|--------|-------------------|----------------|
| **Language** | Multiple | Java/Kotlin | Swift/Objective-C |
| **Cross-platform** | Yes | Android only | iOS only |
| **App modification** | No | Required (test APK) | Required (test target) |
| **Speed** | Moderate | Fast | Fast |
| **Flexibility** | High | Medium | Medium |
| **Best for** | E2E, cross-platform | Android unit/integration | iOS-specific |

---

## **25.2 Appium Architecture**

Understanding Appium's architecture is crucial for debugging issues and optimizing test execution.

### **25.2.1 Architectural Overview**

```
┌─────────────────────────────────────────────────────────────┐
│                    Client Layer                              │
│   (Your Test Script: Python/Java/JS)                        │
│         │                                                    │
│         │ HTTP/JSON (WebDriver Protocol)                     │
│         ▼                                                    │
├─────────────────────────────────────────────────────────────┤
│                  Appium Server                               │
│   (Node.js application)                                      │
│   • Receives WebDriver commands                              │
│   • Routes to appropriate driver                             │
│   • Manages sessions and devices                             │
│         │                                                    │
│         │ Translates to platform-specific commands            │
│         ▼                                                    │
├─────────────────────────────────────────────────────────────┤
│              Platform Drivers                                │
│   ┌──────────────┐    ┌──────────────┐                      │
│   │  UIAutomator2 │    │   XCUITest   │                      │
│   │   (Android)   │    │    (iOS)     │                      │
│   └──────────────┘    └──────────────┘                      │
│   ┌──────────────┐    ┌──────────────┐                      │
│   │    Espresso   │    │   WinAppDriver│                     │
│   │   (Android)   │    │  (Windows)   │                      │
│   └──────────────┘    └──────────────┘                      │
│         │                                                    │
│         ▼                                                    │
├─────────────────────────────────────────────────────────────┤
│              Device/Emulator                                 │
│         (Real Device or Simulator)                          │
└─────────────────────────────────────────────────────────────┘
```

**Key Components:**

1. **Appium Server:** Node.js server that exposes REST API, receives connections from clients, and executes commands on mobile devices
2. **Appium Client:** Language-specific libraries (appium-python-client, java-client) that provide APIs and handle HTTP communication
3. **Platform Drivers:** Translate WebDriver commands to platform-specific automation frameworks:
   - **UIAutomator2:** Google's Android automation framework (default for Android)
   - **Espresso:** Google's newer Android testing framework (faster, more reliable)
   - **XCUITest:** Apple's iOS automation framework (only option for iOS)
4. **JSON Wire Protocol:** Standard format for client-server communication (being deprecated in favor of W3C WebDriver protocol)

### **25.2.2 How Appium Works**

**Step-by-Step Execution Flow:**

```python
# Conceptual flow of an Appium test execution
class AppiumExecutionFlow:
    """
    Understanding how Appium executes commands
    """
    
    def step_1_client_sends_command(self):
        """
        1. Client converts method call to HTTP request
        """
        # Python example: driver.find_element(By.ID, "username")
        # Converts to:
        http_request = {
            "method": "POST",
            "endpoint": "/session/{session_id}/element",
            "body": {
                "using": "id",
                "value": "username"
            }
        }
        return http_request
    
    def step_2_server_receives(self):
        """
        2. Appium Server receives and routes command
        """
        routing_logic = {
            "platformName": "Android",  # Routes to UIAutomator2
            "platformName": "iOS",      # Routes to XCUITest
        }
        # Server validates session and forwards to appropriate driver
    
    def step_3_driver_translates(self):
        """
        3. Platform driver translates to native command
        """
        # For Android (UIAutomator2):
        android_command = {
            "action": "findElement",
            "params": {
                "strategy": "id",
                "selector": "com.example.app:id/username"
            }
        }
        
        # For iOS (XCUITest):
        ios_command = {
            "action": "findElement",
            "params": {
                "using": "accessibility id",
                "value": "username"
            }
        }
        return android_command, ios_command
    
    def step_4_device_executes(self):
        """
        4. Device executes command and returns result
        """
        # UIAutomator2/XCUITest interact with Accessibility/View hierarchy
        # Return element ID or error
    
    def step_5_response_returned(self):
        """
        5. Response propagated back to client
        """
        response = {
            "value": {
                "ELEMENT": "1234-5678-90ab-cdef"  # Unique element ID
            },
            "status": 0  # Success
        }
        # Client converts back to WebElement object
```

### **25.2.3 Session Management**

Appium uses sessions to isolate test executions. Each session gets a unique ID and maintains state.

```python
class SessionManagement:
    """
    Understanding Appium sessions
    """
    
    def session_creation(self):
        """
        Creating a new session
        """
        # Client sends POST /session with capabilities
        capabilities = {
            "platformName": "Android",
            "automationName": "UiAutomator2",
            "deviceName": "Pixel_6",
            "app": "/path/to/app.apk",
            "appPackage": "com.example.app",
            "appActivity": ".MainActivity"
        }
        
        # Server response includes sessionId
        response = {
            "value": {
                "sessionId": "f47ac10b-58cc-4372-a567-0e02b2c3d479",
                "capabilities": { ... }
            }
        }
        
        # All subsequent commands include sessionId in URL:
        # POST /session/f47ac10b-.../element
        # POST /session/f47ac10b-.../element/1234/click
    
    def session_termination(self):
        """
        Ending a session
        """
        # Client sends DELETE /session/{id}
        # Server:
        # 1. Stops app (if noReset=false)
        # 2. Uninstalls helper apps
        # 3. Releases device
        # 4. Cleans up resources
```

---

## **25.3 Environment Setup**

Proper environment setup is critical for Appium success. This section covers step-by-step installation for both Android and iOS platforms.

### **25.3.1 Prerequisites**

**System Requirements:**
- **macOS:** Required for iOS testing (Xcode only runs on Mac)
- **Windows/Linux:** Supported for Android testing only
- **Node.js:** Version 16+ (LTS recommended)
- **Python:** 3.8+ (if using Python client)

### **25.3.2 Installing Appium Server**

**Option 1: Global Installation (Recommended for beginners)**
```bash
# Install Node.js first from https://nodejs.org/

# Install Appium globally
npm install -g appium

# Verify installation
appium --version
# Output: 2.11.0 (or latest)

# Start server
appium
# Server starts on http://localhost:4723 by default
```

**Option 2: Local Installation (Recommended for CI/CD)**
```bash
# Create project directory
mkdir mobile-automation
cd mobile-automation

# Initialize npm
npm init -y

# Install Appium locally
npm install --save-dev appium

# Add to package.json scripts
# "appium": "appium"

# Run via npx
npx appium
```

**Option 3: Desktop App (Deprecated but still used)**
- Appium Desktop provides GUI for inspecting elements
- Download from GitHub releases (though command-line is preferred now)

### **25.3.3 Installing Platform Drivers**

Appium 2.0+ uses a plugin/driver model. You must install drivers separately:

```bash
# Install Android driver (UIAutomator2)
appium driver install uiautomator2

# Install iOS driver (XCUITest) - macOS only
appium driver install xcuitest

# Verify installed drivers
appium driver list
# Output:
# - uiautomator2@2.29.0 [installed]
# - xcuitest@4.35.0 [installed]
```

### **25.3.4 Android SDK Setup**

**Step 1: Install Android Studio**
- Download from https://developer.android.com/studio
- Install Android SDK, SDK Platform-Tools, and Build-Tools

**Step 2: Environment Variables**
```bash
# Add to ~/.bashrc, ~/.zshrc, or Windows System Environment

# Android SDK path
export ANDROID_HOME=$HOME/Library/Android/sdk  # macOS
# export ANDROID_HOME=%LOCALAPPDATA%\Android\Sdk  # Windows

# Add to PATH
export PATH=$PATH:$ANDROID_HOME/platform-tools
export PATH=$PATH:$ANDROID_HOME/tools
export PATH=$PATH:$ANDROID_HOME/tools/bin
```

**Step 3: Verify Setup**
```bash
# Check ADB (Android Debug Bridge)
adb devices
# Output: List of connected devices

# Check Android version
adb --version
```

**Step 4: Create Virtual Device (Optional)**
```bash
# List available system images
sdkmanager --list | grep system-images

# Install image
sdkmanager "system-images;android-34;google_apis;x86_64"

# Create AVD
avdmanager create avd -n Pixel_6_API_34 -k "system-images;android-34;google_apis;x86_64"

# Start emulator
emulator -avd Pixel_6_API_34
```

### **25.3.5 iOS Setup (macOS Only)**

**Step 1: Install Xcode**
- Install from Mac App Store or Apple Developer Portal
- Version must be compatible with target iOS versions

**Step 2: Install Xcode Command Line Tools**
```bash
xcode-select --install

# Verify
xcode-select -p
# Output: /Applications/Xcode.app/Contents/Developer
```

**Step 3: Install ios-deploy (optional but useful)**
```bash
npm install -g ios-deploy
```

**Step 4: Configure Simulators**
- Open Xcode → Preferences → Components
- Download iOS simulators
- Or use command line: `xcrun simctl list devices`

### **25.3.6 Client Library Installation**

**Python:**
```bash
# Install Appium Python client
pip install Appium-Python-Client

# Also install Selenium (dependency)
pip install selenium

# Verify
python -c "from appium import webdriver; print('OK')"
```

**Java (Maven):**
```xml
<dependency>
    <groupId>io.appium</groupId>
    <artifactId>java-client</artifactId>
    <version>9.2.0</version>
</dependency>
```

**JavaScript (WebdriverIO):**
```bash
npm install @wdio/cli --save-dev
npx wdio config
```

### **25.3.7 Verification Script**

Create this verification script to validate your setup:

```python
# setup_verification.py
import subprocess
import sys

def verify_appium():
    """Verify Appium server installation"""
    try:
        result = subprocess.run(['appium', '--version'], 
                              capture_output=True, text=True)
        print(f"✓ Appium installed: {result.stdout.strip()}")
        return True
    except FileNotFoundError:
        print("✗ Appium not found. Install with: npm install -g appium")
        return False

def verify_android():
    """Verify Android SDK"""
    try:
        result = subprocess.run(['adb', '--version'], 
                              capture_output=True, text=True)
        print(f"✓ ADB installed")
        return True
    except FileNotFoundError:
        print("✗ Android SDK not found. Check ANDROID_HOME")
        return False

def verify_drivers():
    """Verify platform drivers"""
    try:
        result = subprocess.run(['appium', 'driver', 'list'], 
                              capture_output=True, text=True)
        if 'uiautomator2' in result.stdout:
            print("✓ UIAutomator2 driver installed")
        if 'xcuitest' in result.stdout:
            print("✓ XCUITest driver installed")
        return True
    except:
        return False

if __name__ == "__main__":
    checks = [verify_appium(), verify_android(), verify_drivers()]
    if all(checks):
        print("\n🎉 Setup complete! Ready to automate.")
    else:
        print("\n⚠️  Please fix issues above before proceeding.")
        sys.exit(1)
```

---

## **25.4 Desired Capabilities**

Desired Capabilities are key-value pairs sent to the Appium server to configure the test session. They tell Appium what type of session to start, which device to use, and which app to test.

### **25.4.1 Common Capabilities**

**Generic Capabilities (All Platforms):**

| Capability | Description | Example |
|------------|-------------|---------|
| `platformName` | Mobile OS platform | `iOS`, `Android`, `Windows` |
| `platformVersion` | OS version | `17.0`, `14.0` |
| `deviceName` | Device name (or simulator name) | `iPhone 15`, `Pixel 6` |
| `automationName` | Driver to use | `XCUITest`, `UiAutomator2`, `Espresso` |
| `app` | Path to app file | `/path/to/app.apk` |
| `noReset` | Don't reset app state | `true`, `false` |
| `fullReset` | Uninstall app before/after | `true`, `false` |
| `newCommandTimeout` | Session timeout in seconds | `300` |

**Android-Specific Capabilities:**

| Capability | Description | Example |
|------------|-------------|---------|
| `appPackage` | Java package of app | `com.example.app` |
| `appActivity` | Activity to launch | `.MainActivity` |
| `appWaitActivity` | Activity to wait for | `.SplashActivity` |
| `autoGrantPermissions` | Auto-accept permissions | `true` |
| `chromedriverExecutable` | Path to ChromeDriver | `/path/to/chromedriver` |

**iOS-Specific Capabilities:**

| Capability | Description | Example |
|------------|-------------|---------|
| `bundleId` | App bundle identifier | `com.example.app` |
| `udid` | Device unique identifier | `auto` or specific UDID |
| `xcodeOrgId` | Apple Developer Team ID | `TEAM123456` |
| `xcodeSigningId` | Signing certificate | `iPhone Developer` |
| `wdaLocalPort` | WebDriverAgent port | `8100` |

### **25.4.2 Complete Configuration Examples**

**Android Real Device:**
```python
from appium import webdriver
from appium.options.android import UiAutomator2Options

def android_real_device_setup():
    """
    Configuration for Android real device testing
    """
    options = UiAutomator2Options()
    
    # Platform
    options.platform_name = 'Android'
    options.platform_version = '14'  # Android 14
    options.device_name = 'Pixel_6'  # Can be any string for real devices
    
    # Automation
    options.automation_name = 'UiAutomator2'
    
    # App configuration
    options.app = '/path/to/app.apk'  # Or use appPackage for installed apps
    options.app_package = 'com.example.app'
    options.app_activity = '.MainActivity'
    
    # Behavior
    options.no_reset = False  # Clear app data before test
    options.auto_grant_permissions = True  # Auto-accept permissions
    
    # Timeouts
    options.new_command_timeout = 300  # 5 minutes
    
    # Create driver
    driver = webdriver.Remote(
        command_executor='http://localhost:4723',
        options=options
    )
    
    return driver
```

**Android Emulator:**
```python
def android_emulator_setup():
    """
    Configuration for Android emulator
    """
    options = UiAutomator2Options()
    
    options.platform_name = 'Android'
    options.platform_version = '14'
    options.device_name = 'emulator-5554'  # AVD name or emulator-5554
    options.automation_name = 'UiAutomator2'
    
    # App
    options.app = '/path/to/app.apk'
    
    # Emulator-specific
    options.avd = 'Pixel_6_API_34'  # Launch this AVD if not running
    options.avd_launch_timeout = 120000  # 2 minutes to wait for boot
    
    driver = webdriver.Remote(
        command_executor='http://localhost:4723',
        options=options
    )
    
    return driver
```

**iOS Simulator:**
```python
from appium.options.ios import XCUITestOptions

def ios_simulator_setup():
    """
    Configuration for iOS Simulator (macOS only)
    """
    options = XCUITestOptions()
    
    options.platform_name = 'iOS'
    options.platform_version = '17.0'
    options.device_name = 'iPhone 15 Pro'
    options.automation_name = 'XCUITest'
    
    # App path
    options.app = '/path/to/MyApp.app'  # .app bundle for simulator
    
    # Simulator specific
    options.no_reset = True  # Keep app installed between sessions
    
    driver = webdriver.Remote(
        command_executor='http://localhost:4723',
        options=options
    )
    
    return driver
```

**iOS Real Device:**
```python
def ios_real_device_setup():
    """
    Configuration for iOS real device (requires Apple Developer account)
    """
    options = XCUITestOptions()
    
    options.platform_name = 'iOS'
    options.platform_version = '17.0'
    options.device_name = 'iPhone'
    options.udid = 'auto'  # Auto-detect, or specify UDID
    
    # App
    options.app = '/path/to/MyApp.ipa'  # Signed IPA for real device
    
    # Signing (required for real devices)
    options.xcode_org_id = '<TEAM_ID>'  # From Apple Developer Portal
    options.xcode_signing_id = 'iPhone Developer'
    
    # WebDriverAgent (intermediary between Appium and device)
    options.wda_local_port = 8100
    
    driver = webdriver.Remote(
        command_executor='http://localhost:4723',
        options=options
    )
    
    return driver
```

### **25.4.3 Capability Strategy for Different Scenarios**

```python
class CapabilityStrategies:
    """
    Different capability configurations for testing scenarios
    """
    
    def fresh_install_scenario(self):
        """
        Clean slate testing - uninstall and reinstall
        """
        options = UiAutomator2Options()
        options.no_reset = False  # Clear data
        options.full_reset = True  # Uninstall before session
        return options
    
    def existing_app_scenario(self):
        """
        Test already installed app (faster, no reinstall)
        """
        options = UiAutomator2Options()
        options.no_reset = True  # Keep data
        options.app_package = 'com.example.app'
        options.app_activity = '.MainActivity'
        # Don't specify 'app' capability (uses installed version)
        return options
    
    def parallel_execution_setup(self):
        """
        Running multiple tests in parallel
        """
        # Test 1
        options1 = UiAutomator2Options()
        options1.device_name = ' emulator-5554'
        options1.system_port = 8200  # Unique port for each session
        
        # Test 2
        options2 = UiAutomator2Options()
        options2.device_name = 'emulator-5556'
        options2.system_port = 8201
        
        return options1, options2
    
    def hybrid_app_setup(self):
        """
        For testing hybrid apps with webviews
        """
        options = UiAutomator2Options()
        options.enable_performance_logging = True
        options.chrome_options = {
            'androidPackage': 'com.example.app',
            'androidUseRunningApp': True
        }
        return options
```

---

## **25.5 Locating Mobile Elements**

Finding elements is the foundation of mobile automation. Mobile apps use different locator strategies than web apps.

### **25.5.1 Mobile Element Hierarchy**

**Android View Hierarchy:**
```
Activity (Window)
└── ViewGroup (LinearLayout, ConstraintLayout)
    └── ViewGroup
        ├── View (Button, TextView, EditText)
        ├── View (ImageView)
        └── ViewGroup
            └── View (TextView - "Login")
```

**iOS View Hierarchy:**
```
Application
└── Window
    └── Other (Container)
        └── Button
        └── TextField
        └── StaticText (label)
```

### **25.5.2 Locator Strategies**

**1. Accessibility ID (Recommended)**
```python
from appium.webdriver.common.appiumby import AppiumBy

# iOS: accessibilityLabel in Xcode
# Android: contentDescription in XML
element = driver.find_element(AppiumBy.ACCESSIBILITY_ID, "login_button")

# Best practice: Ask developers to add accessibility IDs for testability
# iOS: element.accessibilityIdentifier = "login_button"
# Android: android:contentDescription="@string/login_button_desc"
```

**2. ID (Android Resource ID)**
```python
# Android: Resource ID (com.example.app:id/button_login)
# iOS: Not applicable (use Accessibility ID instead)
element = driver.find_element(AppiumBy.ID, "com.example.app:id/username_input")
```

**3. XPath (Powerful but slow)**
```python
# Absolute XPath (fragile, avoid)
element = driver.find_element(AppiumBy.XPATH, 
    "/hierarchy/android.widget.FrameLayout/.../android.widget.Button")

# Relative XPath with attributes (better)
element = driver.find_element(AppiumBy.XPATH, 
    "//android.widget.Button[@text='Login']")

# XPath with contains (flexible)
element = driver.find_element(AppiumBy.XPATH, 
    "//*[contains(@text, 'Welcome')]")

# iOS XPath
element = driver.find_element(AppiumBy.XPATH, 
    "//XCUIElementTypeButton[@name='Login']")
```

**4. Class Name**
```python
# Android: Full class name
buttons = driver.find_elements(AppiumBy.CLASS_NAME, "android.widget.Button")

# iOS: XCUIElementType
text_fields = driver.find_elements(AppiumBy.CLASS_NAME, "XCUIElementTypeTextField")
```

**5. Android UI Automator (Android only, fast)**
```python
# Using UiSelector syntax
element = driver.find_element(AppiumBy.ANDROID_UIAUTOMATOR, 
    'new UiSelector().text("Login").className("android.widget.Button")')

# Complex selectors
element = driver.find_element(AppiumBy.ANDROID_UIAUTOMATOR,
    'new UiSelector().resourceId("com.example.app:id/list").childSelector(new UiSelector().text("Item 1"))')

# Scroll to element
driver.find_element(AppiumBy.ANDROID_UIAUTOMATOR,
    'new UiScrollable(new UiSelector().scrollable(true)).scrollIntoView(new UiSelector().text("Exact Text"))')
```

**6. iOS Class Chain (iOS only)**
```python
# iOS Predicate for complex queries
element = driver.find_element(AppiumBy.IOS_PREDICATE, 
    'name == "Login" AND type == "XCUIElementTypeButton"')

# Class chain (hierarchical)
element = driver.find_element(AppiumBy.IOS_CLASS_CHAIN, 
    '**/XCUIElementTypeTable/XCUIElementTypeCell[`name == "Cell 1"`]/XCUIElementTypeButton')
```

**7. iOS Predicate String (Recommended for iOS)**
```python
# Type-safe, fast queries
element = driver.find_element(AppiumBy.IOS_PREDICATE, 
    'label == "Username" AND visible == 1')

# Partial match
element = driver.find_element(AppiumBy.IOS_PREDICATE, 
    'name BEGINSWITH "Login"')

# Regular expressions
element = driver.find_element(AppiumBy.IOS_PREDICATE, 
    'name MATCHES "^Login.*"')
```

### **25.5.3 Element Interaction**

```python
class MobileInteractions:
    """
    Common mobile element interactions
    """
    
    def text_input(self, driver, locator, text):
        """
        Enter text with mobile-specific handling
        """
        element = driver.find_element(*locator)
        
        # Clear existing text (triple tap for mobile)
        element.clear()
        
        # Enter text
        element.send_keys(text)
        
        # Hide keyboard (important for mobile)
        driver.hide_keyboard()
    
    def tap_element(self, driver, locator):
        """
        Tap (click) element
        """
        element = driver.find_element(*locator)
        element.click()
    
    def get_text(self, driver, locator):
        """
        Get element text (attribute varies by platform)
        """
        element = driver.find_element(*locator)
        
        # Cross-platform text retrieval
        if driver.capabilities['platformName'] == 'Android':
            return element.get_attribute("text")
        else:
            return element.get_attribute("label")  # iOS
    
    def check_visibility(self, driver, locator):
        """
        Check if element is displayed
        """
        try:
            element = driver.find_element(*locator)
            return element.is_displayed()
        except:
            return False
```

---

## **25.6 Mobile Gestures and Interactions**

Mobile apps rely heavily on gestures (tap, swipe, pinch, long press). Appium supports these through the Actions API (W3C standard) and legacy TouchAction.

### **25.6.1 W3C Actions API (Recommended)**

The modern, standardized way to perform gestures:

```python
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions import interaction
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.pointer_input import PointerInput
from selenium.webdriver.common.actions.key_input import KeyInput

class MobileGestures:
    """
    Mobile-specific gestures using W3C Actions
    """
    
    def tap(self, driver, element):
        """
        Single tap on element
        """
        actions = ActionChains(driver)
        actions.click(element).perform()
    
    def double_tap(self, driver, element):
        """
        Double tap
        """
        actions = ActionChains(driver)
        actions.double_click(element).perform()
    
    def long_press(self, driver, element, duration=2000):
        """
        Long press/hold for specified milliseconds
        """
        actions = ActionChains(driver)
        actions.click_and_hold(element).pause(duration/1000).release().perform()
    
    def swipe(self, driver, start_x, start_y, end_x, end_y, duration=1000):
        """
        Swipe from point A to point B
        """
        # Create pointer input device
        finger = PointerInput(interaction.POINTER_TOUCH, "finger")
        
        # Build actions
        actions = ActionBuilder(driver, mouse=finger)
        actions.pointer_action.move_to_location(start_x, start_y)
        actions.pointer_action.pointer_down()
        actions.pointer_action.pause(duration/1000)  # Duration in seconds
        actions.pointer_action.move_to_location(end_x, end_y)
        actions.pointer_action.pointer_up()
        
        actions.perform()
    
    def scroll_to_element(self, driver, element_locator):
        """
        Scroll until element is found
        """
        # Get screen dimensions
        size = driver.get_window_size()
        width = size['width']
        height = size['height']
        
        # Swipe up (scroll down) from center
        start_x = width // 2
        start_y = height * 0.8
        end_y = height * 0.2
        
        max_swipes = 10
        for _ in range(max_swipes):
            try:
                element = driver.find_element(*element_locator)
                if element.is_displayed():
                    return element
            except:
                self.swipe(driver, start_x, start_y, start_x, end_y, 800)
        
        raise Exception("Element not found after scrolling")
    
    def pinch_zoom(self, driver, element, scale=2.0):
        """
        Pinch to zoom (iOS only, complex on Android)
        """
        # Get element center
        location = element.location
        size = element.size
        center_x = location['x'] + size['width'] // 2
        center_y = location['y'] + size['height'] // 2
        
        # Create two fingers
        finger1 = PointerInput(interaction.POINTER_TOUCH, "finger1")
        finger2 = PointerInput(interaction.POINTER_TOUCH, "finger2")
        
        actions = ActionBuilder(driver, mouse=finger1)
        
        # Finger 1: top-left to center
        actions.pointer_action.move_to_location(center_x - 100, center_y - 100)
        actions.pointer_action.pointer_down()
        actions.pointer_action.move_to_location(center_x - 50, center_y - 50)
        actions.pointer_action.pointer_up()
        
        # Note: For multi-touch, need to use multiple PointerInput devices
        # This is simplified - full implementation requires simultaneous actions
        actions.perform()
```

### **25.6.2 Legacy TouchAction (Deprecated but still used)**

```python
from appium.webdriver.common.touch_action import TouchAction

class LegacyGestures:
    """
    Legacy TouchAction class (still works but deprecated)
    """
    
    def tap(self, driver, element):
        TouchAction(driver).tap(element).perform()
    
    def long_press(self, driver, element):
        TouchAction(driver).long_press(element).wait(2000).release().perform()
    
    def swipe(self, driver, start_x, start_y, end_x, end_y):
        TouchAction(driver).press(x=start_x, y=start_y).wait(1000).move_to(x=end_x, y=end_y).release().perform()
```

### **25.6.3 Platform-Specific Gestures**

**Android (UIAutomator2):**
```python
def android_specific_gestures(driver):
    """
    Android-specific gesture commands
    """
    # Open notifications
    driver.open_notifications()
    
    # Press key (hardware buttons)
    driver.press_keycode(4)  # Back button
    driver.press_keycode(3)  # Home button
    driver.press_keycode(24) # Volume up
    
    # Toggle airplane mode (requires permissions)
    driver.toggle_airplane_mode()
```

**iOS (XCUITest):**
```python
def ios_specific_gestures(driver):
    """
    iOS-specific gesture commands
    """
    # Shake device
    driver.shake()
    
    # Lock/unlock
    driver.lock()
    driver.unlock()
    
    # Background app
    driver.background_app(5)  # Background for 5 seconds
    
    # Touch ID/Face ID simulation
    driver.execute_script('mobile: sendBiometricMatch', {'type': 'faceId', 'match': True})
```

---

## **25.7 Hybrid App Testing**

Testing hybrid apps requires switching between native and web contexts to interact with WebView content.

### **25.7.1 Context Switching**

```python
class HybridAppTesting:
    """
    Testing hybrid applications with WebViews
    """
    
    def get_contexts(self, driver):
        """
        List available contexts (NATIVE_APP vs WEBVIEW)
        """
        contexts = driver.contexts
        print(f"Available contexts: {contexts}")
        # Output: ['NATIVE_APP', 'WEBVIEW_com.example.app']
        return contexts
    
    def switch_to_webview(self, driver):
        """
        Switch to WebView context for HTML element interaction
        """
        contexts = driver.contexts
        webview_context = None
        
        for context in contexts:
            if 'WEBVIEW' in context:
                webview_context = context
                break
        
        if webview_context:
            driver.switch_to.context(webview_context)
            print(f"Switched to: {webview_context}")
            # Now you can use standard Selenium locators
            # driver.find_element(By.ID, "username")  # HTML ID, not native ID
        else:
            raise Exception("No WebView context found")
    
    def switch_to_native(self, driver):
        """
        Return to native context
        """
        driver.switch_to.context('NATIVE_APP')
        print("Switched back to native context")
    
    def hybrid_interaction_example(self, driver):
        """
        Example: Login in WebView, verify in Native
        """
        # Native: Click button to open webview
        driver.find_element(AppiumBy.ID, "login_webview_button").click()
        
        # Switch to WebView
        self.switch_to_webview(driver)
        
        # Interact with HTML elements
        driver.find_element(By.ID, "email").send_keys("user@example.com")
        driver.find_element(By.ID, "password").send_keys("password")
        driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
        
        # Wait for navigation or token storage
        
        # Switch back to native
        self.switch_to_native(driver)
        
        # Verify native element shows logged in state
        welcome_msg = driver.find_element(AppiumBy.ACCESSIBILITY_ID, "welcome_message")
        assert "Welcome" in welcome_msg.text
```

### **25.7.2 ChromeDriver Configuration (Android)**

For Android WebView testing, Appium needs ChromeDriver:

```python
def setup_chromedriver():
    """
    Configure ChromeDriver for WebView testing
    """
    options = UiAutomator2Options()
    
    # Option 1: Auto-download matching ChromeDriver
    options.chromedriver_autodownload = True
    
    # Option 2: Specify ChromeDriver path
    options.chromedriver_executable = '/path/to/chromedriver'
    
    # Option 3: Use specific ChromeDriver version
    options.chromedriver_executable_dir = '/path/to/chromedrivers/'
    
    return options
```

---

## **25.8 Real Device Configuration**

Testing on real devices differs from emulators/simulators in several ways.

### **25.8.1 Android Real Device**

**Prerequisites:**
1. Enable Developer Options on device (tap Build Number 7 times)
2. Enable USB Debugging
3. Connect via USB and authorize computer

```python
def android_real_device_config():
    """
    Configuration for Android real device
    """
    options = UiAutomator2Options()
    
    # Find device serial
    # Run: adb devices
    # Output: xxxxxxxx device
    
    options.udid = 'xxxxxxxx'  # Device serial from adb devices
    options.platform_version = '14'  # Must match device
    
    # For installed apps (faster than reinstalling)
    options.app_package = 'com.example.app'
    options.app_activity = '.MainActivity'
    options.no_reset = True
    
    return options
```

### **25.8.2 iOS Real Device**

**Prerequisites:**
1. Apple Developer Account ($99/year)
2. Device registered in Developer Portal
3. Provisioning profile with device UDID
4. WebDriverAgent signed and installed

```python
def ios_real_device_config():
    """
    Configuration for iOS real device (macOS + Xcode required)
    """
    options = XCUITestOptions()
    
    # Get UDID: xcrun xctrace list devices
    options.udid = '00008030-001234567890ABCD'
    
    # Signing (required for real devices)
    options.xcode_org_id = 'TEAM123456'  # Team ID from Apple Developer
    options.xcode_signing_id = 'iPhone Developer'
    
    # App must be signed for device
    options.app = '/path/to/ResignedApp.ipa'
    
    # WebDriverAgent configuration
    options.wda_launch_timeout = 120000  # 2 minutes
    
    return options
```

---

## **25.9 Best Practices and Framework Design**

### **25.9.1 Page Object Model (POM) for Mobile**

```python
from abc import ABC, abstractmethod

class BasePage(ABC):
    """
    Base class for mobile page objects
    """
    def __init__(self, driver):
        self.driver = driver
        self.platform = driver.capabilities['platformName']
    
    def find_element(self, android_locator, ios_locator):
        """
        Platform-agnostic element finder
        """
        if self.platform == 'Android':
            return self.driver.find_element(*android_locator)
        else:
            return self.driver.find_element(*ios_locator)
    
    def wait_for_element(self, locator, timeout=10):
        """
        Explicit wait for element
        """
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        
        return WebDriverWait(self.driver, timeout).until(
            EC.presence_of_element_located(locator)
        )

class LoginPage(BasePage):
    """
    Login page implementation
    """
    # Locators
    ANDROID_USERNAME = (AppiumBy.ID, "com.example.app:id/username")
    IOS_USERNAME = (AppiumBy.ACCESSIBILITY_ID, "username_field")
    
    ANDROID_LOGIN_BTN = (AppiumBy.ID, "com.example.app:id/login")
    IOS_LOGIN_BTN = (AppiumBy.IOS_PREDICATE, 'name == "Login"')
    
    def enter_username(self, username):
        element = self.find_element(self.ANDROID_USERNAME, self.IOS_USERNAME)
        element.clear()
        element.send_keys(username)
        return self
    
    def tap_login(self):
        element = self.find_element(self.ANDROID_LOGIN_BTN, self.IOS_LOGIN_BTN)
        element.click()
        return HomePage(self.driver)  # Page transition

class HomePage(BasePage):
    """
    Home page after login
    """
    def verify_logged_in(self):
        # Implementation
        pass
```

### **25.9.2 Test Setup and Teardown**

```python
import pytest
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.options.ios import XCUITestOptions

@pytest.fixture(scope="function")
def driver(request):
    """
    Pytest fixture for Appium driver management
    """
    platform = request.config.getoption("--platform")
    
    if platform == "android":
        options = UiAutomator2Options()
        options.platform_name = 'Android'
        options.device_name = 'Pixel_6'
        options.app = '/path/to/app.apk'
        options.app_package = 'com.example.app'
        options.app_activity = '.MainActivity'
    else:
        options = XCUITestOptions()
        options.platform_name = 'iOS'
        options.device_name = 'iPhone 15'
        options.app = '/path/to/app.app'
    
    driver = webdriver.Remote('http://localhost:4723', options=options)
    
    yield driver
    
    # Teardown
    driver.quit()

def test_login(driver):
    """
    Example test using fixture
    """
    login_page = LoginPage(driver)
    login_page.enter_username("testuser")
    login_page.enter_password("password")
    home_page = login_page.tap_login()
    assert home_page.is_displayed()
```

### **25.9.3 Handling Waits and Synchronization**

```python
class WaitStrategies:
    """
    Mobile-specific wait strategies
    """
    
    def wait_for_element_visible(self, driver, locator, timeout=10):
        """
        Wait until element is visible
        """
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        
        return WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located(locator)
        )
    
    def wait_for_element_invisible(self, driver, locator, timeout=10):
        """
        Wait for loading spinner to disappear
        """
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        
        return WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located(locator)
        )
    
    def mobile_wait_for_condition(self, driver, condition_func, timeout=10):
        """
        Custom wait for mobile-specific conditions
        """
        import time
        start = time.time()
        while time.time() - start < timeout:
            if condition_func(driver):
                return True
            time.sleep(0.5)
        raise TimeoutError("Condition not met")
```

---

## **25.10 Troubleshooting Common Issues**

```python
class TroubleshootingGuide:
    """
    Common Appium issues and solutions
    """
    
    def issue_session_not_created(self):
        """
        Error: Could not start session
        """
        solutions = [
            "Verify Appium server is running (appium)",
            "Check port 4723 is not blocked",
            "Verify capabilities match device/platform",
            "Check device is connected (adb devices)",
            "Review Appium server logs for specific errors"
        ]
        return solutions
    
    def issue_element_not_found(self):
        """
        Error: NoSuchElementException
        """
        solutions = [
            "Check if element is in WebView (switch context)",
            "Verify element exists in current activity/screen",
            "Add explicit wait (element might load asynchronously)",
            "Check for typos in locator strategy",
            "Use Appium Inspector to verify correct locator",
            "Check if element is inside ScrollView (need to scroll)"
        ]
        return solutions
    
    def issue_webdriveragent(self):
        """
        iOS: WebDriverAgent not starting
        """
        solutions = [
            "Update to latest Appium and drivers",
            "Clear Xcode derived data",
            "Re-sign WebDriverAgent with valid certificate",
            "Check iOS version compatibility with Xcode",
            "Try: xcodebuild clean in WebDriverAgent directory"
        ]
        return solutions
```

---

## **Chapter Summary**

### **Key Takeaways from Chapter 25:**

**Appium Architecture:**
- **Client-Server Model:** Tests (client) send HTTP requests to Appium Server (Node.js), which routes to platform-specific drivers (UIAutomator2 for Android, XCUITest for iOS)
- **WebDriver Protocol:** Standardized JSON over HTTP, enabling cross-platform automation with language-agnostic clients
- **Session Management:** Each test creates a session with unique ID; capabilities configure device, app, and behavior

**Environment Setup:**
- **Installation:** Node.js + Appium server + platform drivers (uiautomator2, xcuitest) + client libraries
- **Android:** Android SDK, platform-tools, ADB; environment variables ANDROID_HOME and PATH
- **iOS:** Xcode (macOS only), command line tools, WebDriverAgent signing for real devices
- **Verification:** Use `appium driver list` and `adb devices` to validate setup

**Desired Capabilities:**
- **Core:** `platformName`, `deviceName`, `app`, `automationName` (UiAutomator2/XCUITest)
- **Android-specific:** `appPackage`, `appActivity`, `autoGrantPermissions`
- **iOS-specific:** `bundleId`, `udid`, `xcodeOrgId` (for real devices), `wdaLocalPort`
- **Strategy:** Use `noReset=True` for speed in development, `fullReset` for clean slate testing

**Element Location:**
- **Preferred:** Accessibility ID (cross-platform, fastest), ID (Android), iOS Predicate (iOS)
- **Avoid:** XPath (slow, brittle) unless necessary for dynamic elements
- **Android-specific:** UI Automator selectors for complex queries and scrolling
- **iOS-specific:** Class Chain and Predicate for type-safe, efficient queries

**Mobile Interactions:**
- **W3C Actions API:** Modern standard for gestures (tap, long press, swipe, pinch)
- **Legacy:** TouchAction (deprecated but still functional)
- **Platform-specific:** Android (keycodes, notifications), iOS (shake, biometrics, background/foreground)

**Hybrid Apps:**
- **Context Switching:** Native (`NATIVE_APP`) vs WebView (`WEBVIEW_xxx`) contexts
- **ChromeDriver:** Required for Android WebView automation; auto-download or specify path
- **Workflow:** Switch to WebView for HTML interaction, back to Native for app chrome

**Real Devices:**
- **Android:** Enable USB debugging, use `udid` capability, faster than emulators for local testing
- **iOS:** Requires Apple Developer account, code signing, WebDriverAgent installation; use `xcodeOrgId` and `xcodeSigningId`
- **Challenges:** Device fragmentation, OS variations, hardware-specific features

**Framework Design:**
- **Page Object Model:** Abstract platform differences with base classes, separate Android/iOS locators
- **Fixtures:** Use pytest/setup methods for driver lifecycle management
- **Waits:** Explicit waits essential for mobile (network latency, animations); avoid implicit waits

**Industry Standards:**
- **W3C WebDriver:** Moving from JSON Wire Protocol to standard WebDriver
- **CI/CD Integration:** Run on device clouds (Sauce Labs, BrowserStack) for parallel execution
- **Best Practices:** Accessibility IDs for testability, avoid XPath, use emulator for CI (speed), real devices for release candidates

---

## **📖 Next Chapter: Chapter 26 - Mobile Testing Tools and Platforms**

Now that you've mastered Appium automation, **Chapter 26** will expand your toolkit to include **specialized mobile testing tools and cloud platforms** that complement your automation framework.

In **Chapter 26**, you'll learn:

- **Espresso and XCUITest:** Native frameworks for Android and iOS that offer superior speed and reliability compared to Appium for specific use cases
- **Detox:** Gray-box testing framework for React Native applications with automatic synchronization
- **Device Cloud Platforms:** Deep dive into BrowserStack, Sauce Labs, AWS Device Farm, and Firebase Test Lab for scalable device coverage
- **Performance Testing Tools:** Firebase Performance Monitoring, Xcode Instruments, and Android Profiler for measuring app performance
- **Visual Testing:** Applitools and Percy for mobile UI regression testing across devices
- **Security Testing Tools:** MobSF (Mobile Security Framework) for automated security analysis of mobile apps
- **Beta Testing Distribution:** TestFlight, Firebase App Distribution, and Play Console for managing beta releases

**Chapter 26** will help you build a comprehensive mobile testing ecosystem that combines **local automation (Appium)**, **native frameworks (Espresso/XCUITest)**, and **cloud-based device labs** to achieve maximum coverage and efficiency.

**Continue to Chapter 26 to complete your mobile testing toolkit and learn to scale your testing across hundreds of real devices!**